In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import seaborn as sns 
from sklearn.ensemble import RandomForestClassifier
from sklearn import datasets
from sklearn import metrics
from sklearn.model_selection import train_test_split

In [15]:
!gdown --id '1DPNEMWj7Xy-CIXad0wX7WT9g58TtY9Nq' --output level-6.csv

Downloading...
From: https://drive.google.com/uc?id=1DPNEMWj7Xy-CIXad0wX7WT9g58TtY9Nq
To: /content/level-6.csv
100% 182k/182k [00:00<00:00, 1.61MB/s]


In [16]:
#Clinical: 0->control 1->dep_M0 2->dep_M3
train = pd.read_csv("level-6.csv")
train.head()

,index,Bacteria;Abditibacteriota;Abditibacteria;Abditibacteriales;Abditibacteriaceae;Abditibacterium,Bacteria;Acidobacteriota;Acidobacteriae;Subgroup_2;__;__,Bacteria;Actinobacteria;Actinobacteria;Actinomycetales;Actinomycetaceae;Actinomyces,Bacteria;Actinobacteriota;Acidimicrobiia;Microtrichales;Ilumatobacteraceae;uncultured,Bacteria;Actinobacteriota;Acidimicrobiia;uncultured;uncultured_forest_soil_bacterium;,Bacteria;Actinobacteriota;Actinobacteria;Actinomycetales;Actinomycetaceae;Actinomyces,Bacteria;Actinobacteriota;Actinobacteria;Actinomycetales;Actinomycetaceae;F0332,Bacteria;Actinobacteriota;Actinobacteria;Actinomycetales;Actinomycetaceae;Varibaculum,Bacteria;Actinobacteriota;Actinobacteria;Bifidobacteriales;Bifidobacteriaceae;Bifidobacterium,Bacteria;Actinobacteriota;Actinobacteria;Bifidobacteriales;Bifidobacteriaceae;Gardnerella,Bacteria;Actinobacteriota;Actinobacteria;Corynebacteriales;Corynebacteriaceae;Corynebacterium,Bacteria;Actinobacteriota;Actinobacteria;Corynebacteriales;Corynebacteriaceae;Lawsonella,Bacteria;Actinobacteriota;Actinobacteria;Corynebacteriales;Dietziaceae;Dietzia,Bacteria;Actinobacteriota;Actinobacteria;Corynebacteriales;Mycobacteriaceae;Mycobacterium,Bacteria;Actinobacteriota;Actinobacteria;Corynebacteriales;Nocardiaceae;Gordonia,Bacteria;Actinobacteriota;Actinobacteria;Corynebacteriales;Nocardiaceae;Rhodococcus,Bacteria;Actinobacteriota;Actinobacteria;Corynebacteriales;Nocardiaceae;Williamsia,Bacteria;Actinobacteriota;Actinobacteria;Frankiales;Geodermatophilaceae;Blastococcus,Bacteria;Actinobacteriota;Actinobacteria;Frankiales;Geodermatophilaceae;Modestobacter,Bacteria;Actinobacteriota;Actinobacteria;Frankiales;Sporichthyaceae;uncultured,Bacteria;Actinobacteriota;Actinobacteria;Frankiales;__;__,Bacteria;Actinobacteriota;Actinobacteria;Frankiales;uncultured;Actinomycetales_bacterium,Bacteria;Actinobacteriota;Actinobacteria;Frankiales;uncultured;__,Bacteria;Actinobacteriota;Actinobacteria;Kineosporiales;Kineosporiaceae;Quadrisphaera,Bacteria;Actinobacteriota;Actinobacteria;Micrococcales;Beutenbergiaceae;Miniimonas,Bacteria;Actinobacteriota;Actinobacteria;Micrococcales;Bogoriellaceae;Georgenia,Bacteria;Actinobacteriota;Actinobacteria;Micrococcales;Brevibacteriaceae;Brevibacterium,Bacteria;Actinobacteriota;Actinobacteria;Micrococcales;Dermabacteraceae;Brachybacterium,Bacteria;Actinobacteriota;Actinobacteria;Micrococcales;Dermabacteraceae;Dermabacter,Bacteria;Actinobacteriota;Actinobacteria;Micrococcales;Dermacoccaceae;Barrientosiimonas,Bacteria;Actinobacteriota;Actinobacteria;Micrococcales;Dermacoccaceae;Dermacoccus,Bacteria;Actinobacteriota;Actinobacteria;Micrococcales;Intrasporangiaceae;Janibacter,Bacteria;Actinobacteriota;Actinobacteria;Micrococcales;Intrasporangiaceae;Lapillicoccus,Bacteria;Actinobacteriota;Actinobacteria;Micrococcales;Intrasporangiaceae;Ornithinicoccus,Bacteria;Actinobacteriota;Actinobacteria;Micrococcales;Intrasporangiaceae;Ornithinimicrobium,Bacteria;Actinobacteriota;Actinobacteria;Micrococcales;Intrasporangiaceae;Serinicoccus,Bacteria;Actinobacteriota;Actinobacteria;Micrococcales;Intrasporangiaceae;__,Bacteria;Actinobacteriota;Actinobacteria;Micrococcales;Microbacteriaceae;Agrococcus,Bacteria;Actinobacteriota;Actinobacteria;Micrococcales;Microbacteriaceae;Amnibacterium,...,Bacteria;Proteobacteria;Gammaproteobacteria;Legionellales;Legionellaceae;Legionella,Bacteria;Proteobacteria;Gammaproteobacteria;Oceanospirillales;Halomonadaceae;Halomonas,Bacteria;Proteobacteria;Gammaproteobacteria;Orbales;Orbaceae;Gilliamella,Bacteria;Proteobacteria;Gammaproteobacteria;Pasteurellales;Pasteurellaceae;Actinobacillus,Bacteria;Proteobacteria;Gammaproteobacteria;Pasteurellales;Pasteurellaceae;Aggregatibacter,Bacteria;Proteobacteria;Gammaproteobacteria;Pasteurellales;Pasteurellaceae;Haemophilus,Bacteria;Proteobacteria;Gammaproteobacteria;Pasteurellales;Pasteurellaceae;Pasteurella,Bacteria;Proteobacteria;Gammaproteobacteria;Pasteurellales;Pasteurellaceae;__,Bacteria;Proteobacteria;Gammaproteob

In [17]:
shortrun = train[train['merged'] < 19250].index
train = train.drop(shortrun,axis=0)
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 130 entries, 0 to 167
Columns: 371 entries, index to Clinical
dtypes: float64(368), int64(2), object(1)
memory usage: 377.8+ KB


In [18]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 130 entries, 0 to 167
Columns: 371 entries, index to Clinical
dtypes: float64(368), int64(2), object(1)
memory usage: 377.8+ KB


In [19]:
CAT_COL = ["index", "Clinical"]

NUM_COL=[]
for i in range(len(train.columns)):
  NUM_COL.append(train.columns[i])
NUM_COL.remove('index')
NUM_COL.remove('Clinical')

cat_col = []
num_col = []
for col in train:
  if col in CAT_COL:
    cat_col.append(col)
  elif col in NUM_COL:
    num_col.append(col)
# for col in cat_col:
#   train[col] = train[col].astype(str)

df_cat = train.loc[:,cat_col] # take all the categorical columns
df_cat = pd.get_dummies(df_cat) # one hot encoding

df_num = train.loc[:,num_col] # take all the numerical columns
df_final = pd.concat([df_cat, df_num], axis=1) # concat categorical/numerical data

In [20]:
df_final.head()

,Clinical,index_SRR10717942,index_SRR10717943,index_SRR10717944,index_SRR10717945,index_SRR10717947,index_SRR10717950,index_SRR10717951,index_SRR10717952,index_SRR10717953,index_SRR10717954,index_SRR10717956,index_SRR10717957,index_SRR10717958,index_SRR10717959,index_SRR10717960,index_SRR10717961,index_SRR10717963,index_SRR10717964,index_SRR10717965,index_SRR10717966,index_SRR10717967,index_SRR10717968,index_SRR10717969,index_SRR10717970,index_SRR10717971,index_SRR10717972,index_SRR10717973,index_SRR10717974,index_SRR10717975,index_SRR10717976,index_SRR10717977,index_SRR10717978,index_SRR10717979,index_SRR10717980,index_SRR10717981,index_SRR10717983,index_SRR10717984,index_SRR10717985,index_SRR10717986,...,Bacteria;Proteobacteria;Gammaproteobacteria;Enterobacterales;Yersiniaceae;Serratia,Bacteria;Proteobacteria;Gammaproteobacteria;Legionellales;Legionellaceae;Legionella,Bacteria;Proteobacteria;Gammaproteobacteria;Oceanospirillales;Halomonadaceae;Halomonas,Bacteria;Proteobacteria;Gammaproteobacteria;Orbales;Orbaceae;Gilliamella,Bacteria;Proteobacteria;Gammaproteobacteria;Pasteurellales;Pasteurellaceae;Actinobacillus,Bacteria;Proteobacteria;Gammaproteobacteria;Pasteurellales;Pasteurellaceae;Aggregatibacter,Bacteria;Proteobacteria;Gammaproteobacteria;Pasteurellales;Pasteurellaceae;Haemophilus,Bacteria;Proteobacteria;Gammaproteobacteria;Pasteurellales;Pasteurellaceae;Pasteurella,Bacteria;Proteobacteria;Gammaproteobacteria;Pasteurellales;Pasteurellaceae;__,Bacteria;Proteobacteria;Gammaproteobacteria;Pseudomonadales;Moraxellaceae;Acinetobacter,Bacteria;Proteobacteria;Gammaproteobacteria;Pseudomonadales;Moraxellaceae;Alkanindiges,Bacteria;Proteobacteria;Gammaproteobacteria;Pseudomonadales;Moraxellaceae;Enhydrobacter,Bacteria;Proteobacteria;Gammaproteobacteria;Pseudomonadales;Moraxellaceae;Moraxella,Bacteria;Proteobacteria;Gammaproteobacteria;Pseudomonadales;Moraxellaceae;Psychrobacter,Bacteria;Proteobacteria;Gammaproteobacteria;Pseudomonadales;Moraxellaceae;__,Bacteria;Proteobacteria;Gammaproteobacteria;Pseudomonadales;Moraxellaceae;uncultured,Bacteria;Proteobacteria;Gammaproteobacteria;Pseudomonadales;Pseudomonadaceae;Pseudomonas,Bacteria;Proteobacteria;Gammaproteobacteria;Pseudomonadales;__;__,Bacteria;Proteobacteria;Gammaproteobacteria;Salinisphaerales;Solimonadaceae;Nevskia,Bacteria;Proteobacteria;Gammaproteobacteria;Thiotrichales;Thiotrichaceae;Thiothrix,Bacteria;Proteobacteria;Gammaproteobacteria;Vibrionales;Vibrionaceae;Photobacterium,Bacteria;Proteobacteria;Gammaproteobacteria;Vibrionales;Vibrionaceae;Vibrio,Bacteria;Proteobacteria;Gammaproteobacteria;Xanthomonadales;Rhodanobacteraceae;Ahniella,Bacteria;Proteobacteria;Gammaproteobacteria;Xanthomonadales;Rhodanobacteraceae;Pseudofulvimonas,Bacteria;Proteobacteria;Gammaproteobacteria;Xanthomonadales;Rhodanobacteraceae;Rhodanobacter,Bacteria;Proteobacteria;Gammaproteobacteria;Xanthomonadales;Xanthomonadaceae;Luteimonas,Bacteria;Proteobacteria;Gammaproteobacteria;Xanthomonadales;Xanthomonadaceae;Lysobacter,Bacteria;Proteobacteria;Gammaproteobacteria;Xanthomonadales;Xanthomonadaceae;Pseudoxanthomonas,Bacteria;Proteobacteria;Gammaproteobacteria;Xanthomonadales;Xanthomonadaceae;Stenotrophomonas,Bacteria;Proteobacteria;Gammaproteobacteria;Xanthomonadales;Xanthomonadaceae;Xanthomonas,Bacteria;Proteobacteria;Gammaproteobacteria;__;__;__,Bacteria;Proteobacteria;__;__;__;__,Bacteria;SAR324_clade(Marine_group_B);uncultured_bacterium;;__;__,Bacteria;Spirochaetota;Spirochaetia;Spirochaetales;Spirochaetaceae;Treponema,Bacteria;Synergistota;Synergistia;Synergistales;Synergistaceae;Fretibacterium,Bacteria;Verrucomicrobiota;Chlamydiae;Chlamydiales;__;__,Bacteria;Verrucomicrobiota;Omnitrophia;Omnitrophales;Omnitrophaceae;Candidatus_Omnitrophus,Bacteria;__;__;__;__;__,Unassigned;__;__;__;__;__,merged
0,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.060593,0.0,0.0,0.000477,0.0,0.084239,0.0,0.0,0.0,0.0,0.013825,0.0,0.0,0.0,0.0,0.000000,

In [21]:
not_select = ["index", "merged"]
train_select = train.drop(not_select,axis=1)
train_select.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 130 entries, 0 to 167
Columns: 369 entries, Bacteria;Abditibacteriota;Abditibacteria;Abditibacteriales;Abditibacteriaceae;Abditibacterium to Clinical
dtypes: float64(368), int64(1)
memory usage: 375.8 KB


In [22]:
cat_col = []
num_col = []
for col in train_select:
  if col in CAT_COL:
    cat_col.append(col)
  elif col in NUM_COL:
    num_col.append(col)

# for col in cat_col:
#   if train_select[col].dtype != "O":
#     # print(col)
#     train_select[col] = train_select[col].astype(str)

df_cat_select = train_select.loc[:,cat_col] # take all the categorical columns
#df_cat_select = pd.get_dummies(df_cat_select) # one hot encoding

df_num_select = train_select.loc[:,num_col] # take all the numerical columns
df_final_select = pd.concat([df_cat_select, df_num_select], axis=1) # concat categorical/numerical data

In [23]:
df_final_select.head()

,Clinical,Bacteria;Abditibacteriota;Abditibacteria;Abditibacteriales;Abditibacteriaceae;Abditibacterium,Bacteria;Acidobacteriota;Acidobacteriae;Subgroup_2;__;__,Bacteria;Actinobacteria;Actinobacteria;Actinomycetales;Actinomycetaceae;Actinomyces,Bacteria;Actinobacteriota;Acidimicrobiia;Microtrichales;Ilumatobacteraceae;uncultured,Bacteria;Actinobacteriota;Acidimicrobiia;uncultured;uncultured_forest_soil_bacterium;,Bacteria;Actinobacteriota;Actinobacteria;Actinomycetales;Actinomycetaceae;Actinomyces,Bacteria;Actinobacteriota;Actinobacteria;Actinomycetales;Actinomycetaceae;F0332,Bacteria;Actinobacteriota;Actinobacteria;Actinomycetales;Actinomycetaceae;Varibaculum,Bacteria;Actinobacteriota;Actinobacteria;Bifidobacteriales;Bifidobacteriaceae;Bifidobacterium,Bacteria;Actinobacteriota;Actinobacteria;Bifidobacteriales;Bifidobacteriaceae;Gardnerella,Bacteria;Actinobacteriota;Actinobacteria;Corynebacteriales;Corynebacteriaceae;Corynebacterium,Bacteria;Actinobacteriota;Actinobacteria;Corynebacteriales;Corynebacteriaceae;Lawsonella,Bacteria;Actinobacteriota;Actinobacteria;Corynebacteriales;Dietziaceae;Dietzia,Bacteria;Actinobacteriota;Actinobacteria;Corynebacteriales;Mycobacteriaceae;Mycobacterium,Bacteria;Actinobacteriota;Actinobacteria;Corynebacteriales;Nocardiaceae;Gordonia,Bacteria;Actinobacteriota;Actinobacteria;Corynebacteriales;Nocardiaceae;Rhodococcus,Bacteria;Actinobacteriota;Actinobacteria;Corynebacteriales;Nocardiaceae;Williamsia,Bacteria;Actinobacteriota;Actinobacteria;Frankiales;Geodermatophilaceae;Blastococcus,Bacteria;Actinobacteriota;Actinobacteria;Frankiales;Geodermatophilaceae;Modestobacter,Bacteria;Actinobacteriota;Actinobacteria;Frankiales;Sporichthyaceae;uncultured,Bacteria;Actinobacteriota;Actinobacteria;Frankiales;__;__,Bacteria;Actinobacteriota;Actinobacteria;Frankiales;uncultured;Actinomycetales_bacterium,Bacteria;Actinobacteriota;Actinobacteria;Frankiales;uncultured;__,Bacteria;Actinobacteriota;Actinobacteria;Kineosporiales;Kineosporiaceae;Quadrisphaera,Bacteria;Actinobacteriota;Actinobacteria;Micrococcales;Beutenbergiaceae;Miniimonas,Bacteria;Actinobacteriota;Actinobacteria;Micrococcales;Bogoriellaceae;Georgenia,Bacteria;Actinobacteriota;Actinobacteria;Micrococcales;Brevibacteriaceae;Brevibacterium,Bacteria;Actinobacteriota;Actinobacteria;Micrococcales;Dermabacteraceae;Brachybacterium,Bacteria;Actinobacteriota;Actinobacteria;Micrococcales;Dermabacteraceae;Dermabacter,Bacteria;Actinobacteriota;Actinobacteria;Micrococcales;Dermacoccaceae;Barrientosiimonas,Bacteria;Actinobacteriota;Actinobacteria;Micrococcales;Dermacoccaceae;Dermacoccus,Bacteria;Actinobacteriota;Actinobacteria;Micrococcales;Intrasporangiaceae;Janibacter,Bacteria;Actinobacteriota;Actinobacteria;Micrococcales;Intrasporangiaceae;Lapillicoccus,Bacteria;Actinobacteriota;Actinobacteria;Micrococcales;Intrasporangiaceae;Ornithinicoccus,Bacteria;Actinobacteriota;Actinobacteria;Micrococcales;Intrasporangiaceae;Ornithinimicrobium,Bacteria;Actinobacteriota;Actinobacteria;Micrococcales;Intrasporangiaceae;Serinicoccus,Bacteria;Actinobacteriota;Actinobacteria;Micrococcales;Intrasporangiaceae;__,Bacteria;Actinobacteriota;Actinobacteria;Micrococcales;Microbacteriaceae;Agrococcus,Bacteria;Actinobacteriota;Actinobacteria;Micrococcales;Microbacteriaceae;Amnibacterium,...,Bacteria;Proteobacteria;Gammaproteobacteria;Enterobacterales;Erwiniaceae;__,Bacteria;Proteobacteria;Gammaproteobacteria;Enterobacterales;Yersiniaceae;Serratia,Bacteria;Proteobacteria;Gammaproteobacteria;Legionellales;Legionellaceae;Legionella,Bacteria;Proteobacteria;Gammaproteobacteria;Oceanospirillales;Halomonadaceae;Halomonas,Bacteria;Proteobacteria;Gammaproteobacteria;Orbales;Orbaceae;Gilliamella,Bacteria;Proteobacteria;Gammaproteobacteria;Pasteurellales;Pasteurellaceae;Actinobacillus,Bacteria;Proteobacteria;Gammaproteobacteria;Pasteurellales;Pasteurellaceae;Aggregatibacter,Bacteria;Proteobacteria;Gammaproteobacteria;Pasteurellales;Pasteurellaceae;Haemophilus,Bacteria;Proteobacteria;Gammaproteobact

#Calculate correlation

In [24]:
x = df_final_select.values
correlation_matrix = np.corrcoef(x.T)
print(correlation_matrix)
print(correlation_matrix[0])

[[ 1.          0.003295    0.003295   ...         nan -0.10154962
   0.00271476]
 [ 0.003295    1.         -0.00775194 ...         nan -0.016869
  -0.0077955 ]
 [ 0.003295   -0.00775194  1.         ...         nan -0.016869
  -0.0077955 ]
 ...
 [        nan         nan         nan ...         nan         nan
          nan]
 [-0.10154962 -0.016869   -0.016869   ...         nan  1.
  -0.01695326]
 [ 0.00271476 -0.0077955  -0.0077955  ...         nan -0.01695326
   1.        ]]
[ 1.00000000e+00  3.29500470e-03  3.29500470e-03  1.10382658e-01
             nan  1.04358852e-01 -1.38941717e-01 -5.94985305e-02
             nan -1.48808341e-01  3.40112038e-02 -1.56330818e-01
  1.22452881e-01             nan  5.94005703e-02  8.05475554e-02
 -6.73407178e-02 -1.03792648e-01  2.12136856e-02  1.10382658e-01
             nan             nan  1.10382658e-01 -7.49669361e-02
  3.29500470e-03  1.10382658e-01  1.10382658e-01  7.40335970e-02
  9.17103829e-03  1.10382658e-01  1.10382658e-01             nan


/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:2559: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:2560: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


In [25]:
columns = df_final_select.columns
columns_not_select = []
for i in range(len(correlation_matrix[0])):
  if correlation_matrix[0][i] <= 0.1 and correlation_matrix[0][i] >= 0:
    columns_not_select.append(columns[i])
  elif correlation_matrix[0][i] >= -0.1 and correlation_matrix[0][i] <= 0:
    columns_not_select.append(columns[i])
  elif i==0 or np.isnan(correlation_matrix[0][i]):
    columns_not_select.append(columns[i]) 
print(len(columns_not_select))
df_final_select2 = df_final_select.drop(columns_not_select,axis=1)

207


In [26]:
# df_final_select2 = df_final_select2.drop('Clinical',axis=1)

In [27]:
df_final_select2.head()

,Bacteria;Actinobacteria;Actinobacteria;Actinomycetales;Actinomycetaceae;Actinomyces,Bacteria;Actinobacteriota;Acidimicrobiia;uncultured;uncultured_forest_soil_bacterium;,Bacteria;Actinobacteriota;Actinobacteria;Actinomycetales;Actinomycetaceae;Actinomyces,Bacteria;Actinobacteriota;Actinobacteria;Bifidobacteriales;Bifidobacteriaceae;Bifidobacterium,Bacteria;Actinobacteriota;Actinobacteria;Corynebacteriales;Corynebacteriaceae;Corynebacterium,Bacteria;Actinobacteriota;Actinobacteria;Corynebacteriales;Corynebacteriaceae;Lawsonella,Bacteria;Actinobacteriota;Actinobacteria;Corynebacteriales;Nocardiaceae;Williamsia,Bacteria;Actinobacteriota;Actinobacteria;Frankiales;Geodermatophilaceae;Modestobacter,Bacteria;Actinobacteriota;Actinobacteria;Frankiales;uncultured;Actinomycetales_bacterium,Bacteria;Actinobacteriota;Actinobacteria;Micrococcales;Beutenbergiaceae;Miniimonas,Bacteria;Actinobacteriota;Actinobacteria;Micrococcales;Bogoriellaceae;Georgenia,Bacteria;Actinobacteriota;Actinobacteria;Micrococcales;Dermabacteraceae;Dermabacter,Bacteria;Actinobacteriota;Actinobacteria;Micrococcales;Dermacoccaceae;Barrientosiimonas,Bacteria;Actinobacteriota;Actinobacteria;Micrococcales;Intrasporangiaceae;Janibacter,Bacteria;Actinobacteriota;Actinobacteria;Micrococcales;Intrasporangiaceae;Lapillicoccus,Bacteria;Actinobacteriota;Actinobacteria;Micrococcales;Intrasporangiaceae;Ornithinimicrobium,Bacteria;Actinobacteriota;Actinobacteria;Micrococcales;Intrasporangiaceae;Serinicoccus,Bacteria;Actinobacteriota;Actinobacteria;Micrococcales;Microbacteriaceae;Agrococcus,Bacteria;Actinobacteriota;Actinobacteria;Micrococcales;Microbacteriaceae;Curtobacterium,Bacteria;Actinobacteriota;Actinobacteria;Micrococcales;Microbacteriaceae;Rhodoluna,Bacteria;Actinobacteriota;Actinobacteria;Micrococcales;Microbacteriaceae;__,Bacteria;Actinobacteriota;Actinobacteria;Micrococcales;Micrococcaceae;Micrococcus,Bacteria;Actinobacteriota;Actinobacteria;Micromonosporales;Micromonosporaceae;Micromonospora,Bacteria;Actinobacteriota;Actinobacteria;Propionibacteriales;Propionibacteriaceae;Microlunatus,Bacteria;Actinobacteriota;Actinobacteria;Propionibacteriales;Propionibacteriaceae;Micropruina,Bacteria;Actinobacteriota;Actinobacteria;Propionibacteriales;Propionibacteriaceae;Propioniciclava,Bacteria;Actinobacteriota;Actinobacteria;Pseudonocardiales;Pseudonocardiaceae;Crossiella,Bacteria;Actinobacteriota;Actinobacteria;Pseudonocardiales;Pseudonocardiaceae;Saccharopolyspora,Bacteria;Actinobacteriota;Actinobacteria;Streptomycetales;Streptomycetaceae;Streptomyces,Bacteria;Actinobacteriota;Actinobacteria;Streptosporangiales;Streptosporangiales_Incertae_Sedis;Motilibacter,Bacteria;Actinobacteriota;Coriobacteriia;Coriobacteriales;Atopobiaceae;Atopobium,Bacteria;Actinobacteriota;Rubrobacteria;Rubrobacterales;Rubrobacteriaceae;Rubrobacter,Bacteria;Bacteroidota;Bacteroidia;Bacteroidales;Bacteroidaceae;Bacteroides,Bacteria;Bacteroidota;Bacteroidia;Bacteroidales;Muribaculaceae;Muribaculum,Bacteria;Bacteroidota;Bacteroidia;Bacteroidales;Porphyromonadaceae;Porphyromonas,Bacteria;Bacteroidota;Bacteroidia;Bacteroidales;Prevotellaceae;Prevotella,Bacteria;Bacteroidota;Bacteroidia;Chitinophagales;Chitinophagaceae;Flavihumibacter,Bacteria;Bacteroidota;Bacteroidia;Cytophagales;Spirosomaceae;Pseudarcicella,Bacteria;Bacteroidota;Bacteroidia;Flavobacteriales;Flavobacteriaceae;Capnocytophaga,Bacteria;Bacteroidota;Bacteroidia;Flavobacteriales;Flavobacteriaceae;Flavobacterium,...,Bacteria;Proteobacteria;Alphaproteobacteria;Rhodospirillales;uncultured;metagenome,Bacteria;Proteobacteria;Alphaproteobacteria;Sphingomonadales;Sphingomonadaceae;Novosphingobium,Bacteria;Proteobacteria;Alphaproteobacteria;Sphingomonadales;Sphingomonadaceae;Sphingobium,Bacteria;Proteobacteria;Alphaproteobacteria;__;__;__,Bacteria;Proteobacteria;Alphaproteobacteria;uncultured;Prosthecomicrobium_sp._enrichment_culture_clone_A113;,Bacteria;Proteobacteria;Gammaproteobacteria;Acidithiobacillales;Acidithiobacillaceae;Acidithiobacillus,Bacteri

In [28]:
df_final_select2.shape

(130, 162)

#Random Forest Classifier

In [29]:
#Use RandomForestClassifier to predict Clinical
x = df_final_select2
y = train["Clinical"]
# y = np.array(y,dtype=int)
X_train,X_test,y_train,y_test = train_test_split(x,y,test_size=0.4,random_state=0)

#RandomForest
rfc = RandomForestClassifier()
#rfc=RandomForestClassifier(n_estimators=100,n_jobs = -1,random_state =50, min_samples_leaf = 10)
rfc.fit(X_train,y_train)
y_predict = rfc.predict(X_train)
score_rfc = rfc.score(X_test,y_test)
print("Random Forest Accuracy = ",score_rfc*100," %")

Random Forest Accuracy =  38.46153846153847  %


In [30]:
from sklearn.model_selection import KFold
x = df_final_select2
y = train["Clinical"]
kf = KFold(n_splits=5)
best_accuracy = 0
for train_index , test_index in kf.split(x):
    X_train, X_test, y_train, y_test = x.iloc[train_index], x.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
    rfc = RandomForestClassifier()
    rfc.fit(X_train,y_train)
    y_predict = rfc.predict(X_train)
    accuracy = rfc.score(X_test,y_test)
    print("Random Forest Accuracy = ",accuracy*100,"%")
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_rfc = rfc
print("Best Accuracy = ",best_accuracy*100,"%")

Random Forest Accuracy =  50.0 %
Random Forest Accuracy =  34.61538461538461 %
Random Forest Accuracy =  30.76923076923077 %
Random Forest Accuracy =  23.076923076923077 %
Random Forest Accuracy =  30.76923076923077 %
Best Accuracy =  50.0 %


In [31]:
#random forest final accuracy
x_train_new = x
y_train_new = train["Clinical"]
y_train_new = y_train_new.reset_index(drop=True)

y_pred = best_rfc.predict(x_train_new)
count = 0

for i in range(y_pred.shape[0]):
    if y_pred[i] != y_train_new[i]:
        count += 1
    # print(y_pred[i],y_train_new[i])
rfc_accuracy = 1-count/y_pred.shape[0]
print("Random Forest Accuracy = ",rfc_accuracy*100,"%")

Random Forest Accuracy =  90.0 %


#SVM

In [32]:
from sklearn import svm
#Use SVM to predict Clinical
x = df_final_select2
y = train["Clinical"]
# y = np.array(y,dtype=int)
X_train,X_test,y_train,y_test = train_test_split(x,y,test_size=0.4,random_state=0)

clf = svm.SVC()
clf.fit(X_train,y_train)
y_predict = clf.predict(X_train)
score_clf = clf.score(X_test,y_test)
print("SVM Accuracy = ",score_clf*100," %")

SVM Accuracy =  30.76923076923077  %


In [33]:
from sklearn.model_selection import KFold
x = df_final_select2
y = train["Clinical"]
kf = KFold(n_splits=5)
best_accuracy = 0
for train_index , test_index in kf.split(x):
    X_train, X_test, y_train, y_test = x.iloc[train_index], x.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
    clf = svm.SVC()
    clf.fit(X_train,y_train)
    y_predict = clf.predict(X_train)
    accuracy = clf.score(X_test,y_test)
    print("SVM Accuracy = ",accuracy*100,"%")
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_clf = clf
print("Best Accuracy = ",best_accuracy*100,"%")

SVM Accuracy =  23.076923076923077 %
SVM Accuracy =  26.923076923076923 %
SVM Accuracy =  11.538461538461538 %
SVM Accuracy =  15.384615384615385 %
SVM Accuracy =  30.76923076923077 %
Best Accuracy =  30.76923076923077 %


In [34]:
#SVM final accuracy
x_train_new = x
y_train_new = train["Clinical"]
y_train_new = y_train_new.reset_index(drop=True)

y_pred = best_clf.predict(x_train_new)
count = 0

for i in range(y_pred.shape[0]):
    # print(y_pred[i],y_train_new[i])
    if y_pred[i] != y_train_new[i]:
        count += 1
clf_accuracy = 1-count/y_pred.shape[0]
print("SVM Accuracy = ",clf_accuracy*100,"%")

SVM Accuracy =  52.3076923076923 %


#Neural network MLPClassifier

In [35]:
from sklearn.neural_network import MLPClassifier
#Use Neural Network MLPClassifier to predict Clinical
x = df_final_select2
y = train["Clinical"]
# y = np.array(y,dtype=int)
X_train,X_test,y_train,y_test = train_test_split(x,y,test_size=0.4,random_state=0)

nnclf = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(50, 30), random_state=1, max_iter=2000)
nnclf.fit(X_train,y_train)
y_predict = nnclf.predict(X_train)
score_nnclf = nnclf.score(X_test,y_test)
print("Neural Network Accuracy = ",score_nnclf*100," %")

Neural Network Accuracy =  44.230769230769226  %


In [36]:
from sklearn.model_selection import KFold
x = df_final_select2
y = train["Clinical"]
kf = KFold(n_splits=5)
best_accuracy = 0
for train_index , test_index in kf.split(x):
    X_train, X_test, y_train, y_test = x.iloc[train_index], x.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
    nnclf = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(50, 30), random_state=1, max_iter=2000)
    nnclf.fit(X_train,y_train)
    y_predict = nnclf.predict(X_train)
    accuracy = nnclf.score(X_test,y_test)
    print("NN Accuracy = ",accuracy*100,"%")
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_nnclf = nnclf
print("NN Accuracy = ",best_accuracy*100,"%")

NN Accuracy =  38.46153846153847 %
NN Accuracy =  34.61538461538461 %
NN Accuracy =  42.30769230769231 %
NN Accuracy =  26.923076923076923 %
NN Accuracy =  26.923076923076923 %
NN Accuracy =  42.30769230769231 %


In [37]:
#NN final accuracy
x_train_new = x
y_train_new = train["Clinical"]
y_train_new = y_train_new.reset_index(drop=True)

y_pred = best_nnclf.predict(x_train_new)
count = 0

for i in range(y_pred.shape[0]):
    # print(y_pred[i],y_train_new[i])
    if y_pred[i] != y_train_new[i]:
        count += 1
nnclf_accuracy = 1-count/y_pred.shape[0]
print("NN Accuracy = ",nnclf_accuracy*100,"%")

NN Accuracy =  85.38461538461539 %


#Logistic Regression

In [38]:
from sklearn.linear_model import LogisticRegression
#Use Logistic Regression to predict Clinical
x = df_final_select2
y = train["Clinical"]
# y = np.array(y,dtype=int)
X_train,X_test,y_train,y_test = train_test_split(x,y,test_size=0.4,random_state=0)

logclf = LogisticRegression(random_state=0).fit(X_train,y_train)
logclf.predict(X_train)
logclf.predict_proba(X_train)
score_logclf = logclf.score(X_test,y_test)
print("Logistic Regression Accuracy = ",score_logclf*100," %")

Logistic Regression Accuracy =  25.0  %


In [39]:
from sklearn.model_selection import KFold
x = df_final_select2
y = train["Clinical"]
kf = KFold(n_splits=5)
best_accuracy = 0
for train_index , test_index in kf.split(x):
    X_train, X_test, y_train, y_test = x.iloc[train_index], x.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
    logclf = LogisticRegression(random_state=0).fit(X_train,y_train)
    y_predict = logclf.predict(X_train)
    accuracy = logclf.score(X_test,y_test)
    print("Logistic Regression Accuracy = ",accuracy*100,"%")
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_logclf = logclf
print("Logistic Regression Accuracy = ",best_accuracy*100,"%")

Logistic Regression Accuracy =  7.6923076923076925 %
Logistic Regression Accuracy =  26.923076923076923 %
Logistic Regression Accuracy =  3.8461538461538463 %
Logistic Regression Accuracy =  15.384615384615385 %
Logistic Regression Accuracy =  19.230769230769234 %
Logistic Regression Accuracy =  26.923076923076923 %


In [40]:
#Logistic final accuracy
x_train_new = x
y_train_new = train["Clinical"]
y_train_new = y_train_new.reset_index(drop=True)

y_pred = best_logclf.predict(x_train_new)
count = 0

for i in range(y_pred.shape[0]):
    # print(y_pred[i],y_train_new[i])
    if y_pred[i] != y_train_new[i]:
        count += 1
logclf_accuracy = 1-count/y_pred.shape[0]
print("Logistic Regression Accuracy = ",logclf_accuracy*100,"%")

Logistic Regression Accuracy =  42.307692307692314 %


#DNN

In [41]:
import os
import numpy as np
import keras as K
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
os.environ['TF_CPP_MIN_LOG_LEVEL']= '2'

In [42]:
target_var = 'Clinical'
features = list(df_final_select2.columns)
# 目標變量的類別
Class = train[target_var].unique()
# 目標變量的類別字典
Class_dict = dict(zip(Class, range(len(Class))))
# 增加一列target, 將目標變量進行編碼
train['target'] = train[target_var].apply(lambda x: Class_dict[x])
# 對目標變量進行0-1編碼(One-hot Encoding)
lb = LabelBinarizer()
lb.fit(list(Class_dict.values()))
transformed_labels = lb.transform(train['target'])
y_bin_labels = []  # 對多分類進行0-1編碼的變量
for i in range(transformed_labels.shape[1]):
    y_bin_labels.append( 'y' + str(i))
    train['y' + str(i)] = transformed_labels[:, i]
# 將數據集分為訓練集和測試集
train_x, test_x, train_y, test_y = train_test_split(df_final_select2[features], train[y_bin_labels], train_size=0.6, test_size=0.4, random_state=0)

In [43]:
np.random.seed(5)
# model
init = K.initializers.glorot_uniform(seed=1)
optimizer = K.optimizers.Adam()
model = K.models.Sequential()
model.add(K.layers.Dense(units=1024, input_dim=162, kernel_initializer=init, activation='relu'))
model.add(K.layers.Dense(units=2048, kernel_initializer=init, activation= 'relu'))
model.add(K.layers.Dense(units=1024, kernel_initializer=init, activation= 'relu'))
model.add(K.layers.Dense(units=3, kernel_initializer=init, activation='relu'))
model.compile(loss = 'categorical_crossentropy', optimizer = optimizer, metrics = ['accuracy'])

In [44]:
# 訓練模型
b_size = 1
max_epochs = 50
print("Starting training ")
h = model.fit(train_x, train_y, batch_size=b_size, epochs=max_epochs, shuffle=True, verbose=1)
print("Training finished \n")

Starting training 
Epoch 1/50
78/78 [==============================] - 15s 3ms/step - loss: 9.7070 - accuracy: 0.3485
Epoch 2/50
78/78 [==============================] - 0s 3ms/step - loss: 7.8576 - accuracy: 0.5125
Epoch 3/50
78/78 [==============================] - 0s 3ms/step - loss: 10.4699 - accuracy: 0.3504
Epoch 4/50
78/78 [==============================] - 0s 3ms/step - loss: 10.0338 - accuracy: 0.3775
Epoch 5/50
78/78 [==============================] - 0s 3ms/step - loss: 10.4767 - accuracy: 0.3500
Epoch 6/50
78/78 [==============================] - 0s 3ms/step - loss: 9.6463 - accuracy: 0.4015
Epoch 7/50
78/78 [==============================] - 0s 3ms/step - loss: 9.3815 - accuracy: 0.4180
Epoch 8/50
78/78 [==============================] - 0s 3ms/step - loss: 11.2435 - accuracy: 0.3024
Epoch 9/50
78/78 [==============================] - 0s 3ms/step - loss: 9.7107 - accuracy: 0.3975
Epoch 10/50
78/78 [==============================] - 0s 3ms/step - loss: 10.1582 - accuracy: 0

In [45]:
eval = model.evaluate(test_x, test_y, verbose=0)
print("Evaluation on test data: loss = %0.6f accuracy = %0.2f%% \n" % (eval[0], eval[1] * 100))

Evaluation on test data: loss = 12.398535 accuracy = 23.08% 



#PH-CNN
##reference:
###https://github.com/dfioravanti/phylogenetic-cnn
###https://www.tensorflow.org/tutorials/images/cnn

In [46]:
from tensorflow.keras import datasets, layers, models

##Reshape input
剩下162筆資料，把二維資料reshape成四維

In [47]:
target_var = 'Clinical'
features = list(df_final_select2.columns)
# 目標變量的類別
Class = train[target_var].unique()
# 目標變量的類別字典
Class_dict = dict(zip(Class, range(len(Class))))
# 增加一列target, 將目標變量進行編碼
train['target'] = train[target_var].apply(lambda x: Class_dict[x])
# 對目標變量進行0-1編碼(One-hot Encoding)
lb = LabelBinarizer()
lb.fit(list(Class_dict.values()))
transformed_labels = lb.transform(train['target'])
y_bin_labels = []  # 對多分類進行0-1編碼的變量
for i in range(transformed_labels.shape[1]):
    y_bin_labels.append( 'y' + str(i))
    train['y' + str(i)] = transformed_labels[:, i]
# 將數據集分為訓練集和測試集
train_x, test_x, train_y, test_y = train_test_split(df_final_select2[features], train[y_bin_labels], train_size=0.6, test_size=0.4, random_state=0)

In [48]:
#reshape train_x
train_x = train_x.values.reshape(78, 3, 3, 18)
train_x = train_x.reshape(78, 3, 3, 18)
test_x = test_x.values.reshape(52, 3, 3, 18)
test_x = test_x.reshape(52, 3, 3, 18)

# train_x = train_x.reshape(78, 2, 2, 64)
# test_x = test_x.reshape(52, 2, 2, 64)

## Model(Same as paper)

In [49]:
model = K.models.Sequential()
model.add(K.layers.Conv2D(18, (1, 1), activation='relu', input_shape=train_x.shape[1:]))
model.add(K.layers.MaxPooling2D((2, 2)))
model.add(K.layers.Flatten())
model.add(K.layers.Dense(64, activation= 'relu'))
model.add(K.layers.Dropout(0.25))
model.add(K.layers.Dense(3,activation='sigmoid'))
# model.add(K.layers.Conv2D(64, (1, 1), activation='sigmoid'))
# model.add(K.layers.MaxPooling2D((2, 2)))
# model.add(K.layers.Conv2D(64, (3, 3), activation='relu'))

In [50]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 3, 3, 18)          342       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 1, 1, 18)          0         
_________________________________________________________________
flatten (Flatten)            (None, 18)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 64)                1216      
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 195       
Total params: 1,753
Trainable params: 1,753
Non-trainable params: 0
____________________________________________________

In [51]:
opt = K.optimizers.Adam(learning_rate=0.0005)
# opt = K.optimizers.Adam(lr=0.0005)
model.compile(optimizer=opt, loss='categorical_crossentropy',
        metrics=['accuracy'])

h2 = model.fit(train_x, train_y, epochs=1, batch_size=1,
      validation_data=(test_x, test_y))

78/78 [==============================] - 30s 8ms/step - loss: 1.0976 - accuracy: 0.3786 - val_loss: 1.0996 - val_accuracy: 0.3077


##Model 2
Activation = relu  
No Dropout

In [52]:
model = K.models.Sequential()
model.add(K.layers.Conv2D(64, (1, 1), activation='relu', input_shape=train_x.shape[1:]))
model.add(K.layers.MaxPooling2D((2, 2)))
model.add(K.layers.Flatten())
model.add(K.layers.Dense(64, activation= 'relu'))
# model.add(K.layers.Dropout(0.25))
model.add(K.layers.Dense(3,activation='relu'))

In [53]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 3, 3, 64)          1216      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 1, 1, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 64)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_7 (Dense)              (None, 3)                 195       
Total params: 5,571
Trainable params: 5,571
Non-trainable params: 0
_________________________________________________________________


In [54]:
opt = K.optimizers.Adam(learning_rate=0.002, decay=1e-2)
model.compile(optimizer=opt, loss='categorical_crossentropy',
        metrics=['accuracy'])

h2 = model.fit(train_x, train_y, epochs=1, batch_size=1,
      validation_data=(test_x, test_y))

78/78 [==============================] - 2s 7ms/step - loss: nan - accuracy: 0.2912 - val_loss: 8.9889 - val_accuracy: 0.4423
